In [3]:
import pandas as pd
import json
import re
import ast
import numpy as np
from datetime import datetime
import os


In [4]:
# load the df
with open('papers/latest_papers.json', 'r') as f:
    data = json.load(f)
df = pd.DataFrame(data)

In [5]:
df.columns

Index(['id', 'abstract', 'authors', 'citationCount',
       'citationCount_normalized', 'citations', 'classification_ids',
       'concepts', 'doi', 'embedding', 'isOpenAccess',
       'keyword_classification_ids', 'language', 'paperId', 'publication_date',
       'relevance_score', 'text', 'title', 'url', 'x', 'y', 'year'],
      dtype='object')

In [6]:
df.head()

,id,abstract,authors,citationCount,citationCount_normalized,citations,classification_ids,concepts,doi,embedding,...,language,paperId,publication_date,relevance_score,text,title,url,x,y,year
0,https://openalex.org/W1002897494,Abstract Carbon Capture and Storage is develop...,"[[https://openalex.org/A5054450376, Yasin Ghol...",10.0,NaN,"[https://openalex.org/W1753239825, https://ope...","[[numerical pressure-decay method, 4.1, 0.77],...","[{'id': 'https://openalex.org/C10899652', 'wik...",https://doi.org/10.1016/j.molliq.2015.06.060,"[-0.4225695729255676, 0.062300171703100204, 0....",...,en,https://openalex.org/W1002897494,2015-11-01,21.742895,Title: Suggesting a numerical pressure-decay m...,Suggesting a numerical pressure-decay method f...,https://doi.org/10.1016/j.molliq.2015.06.060,-42.265751,4.114934,2015.0
1,https://openalex.org/W1010274275,Searching for efficient solid sorbents for CO2...,"[[https://openalex.org/A5005055697, Guoping Ga...",38.0,NaN,"[https://openalex.org/W1966750682, https://ope...","[[CO2 adsorption, 2.9, 0.81], [separation, 4.2...","[{'id': 'https://openalex.org/C162862793', 'wi...",https://doi.org/10.1016/j.commatsci.2015.06.005,"[-0.8119574189186096, -0.12326264381408691, 0....",...,en,https://openalex.org/W1010274275,2015-10-01,21.940535,Title: Modelling CO 2 adsorption and separatio...,Modelling CO 2 adsorption and separation on ex...,https://doi.org/10.1016/j.commatsci.2015.06.005,-15.913016,-9.927729,2015.0
2,https://openalex.org/W1012054217,The gaseous carbon dioxide incorporation react...,"[[https://openalex.org/A5044055674, Chika Haya...",21.0,NaN,"[https://openalex.org/W1973556473, https://ope...","[[Cobalt-Catalyzed Reductive Carboxylation, 10...","[{'id': 'https://openalex.org/C185592680', 'wi...",https://doi.org/10.1246/bcsj.20150043,"[-0.905866265296936, -0.24191510677337646, 1.2...",...,en,https://openalex.org/W1012054217,2015-06-15,21.467160,Title: Cobalt-Catalyzed Reductive Carboxylatio...,"Cobalt-Catalyzed Reductive Carboxylation of α,...",https://doi.org/10.1246/bcsj.20150043,-12.920057,-1.328741,2015.0
3,https://openalex.org/W101897731,None,"[[https://openalex.org/A5030437606, Teng Ben],...",1.0,NaN,"[https://openalex.org/W656376090, https://open...","[[Carbon Dioxide Capture, 2.1, 0.79], [Porous ...","[{'id': 'https://openalex.org/C150394285', 'wi...",https://doi.org/10.1007/978-3-642-54646-4_4,"[-0.048764150589704514, -0.2366003841161728, 0...",...,en,https://openalex.org/W101897731,2014-01-01,22.707438,Title: Carbon Dioxide Capture in Porous Aromat...,Carbon Dioxide Capture in Porous Aromatic Fram...,https://doi.org/10.1007/978-3-642-54646-4_4,-9.399205,-15.376349,2014.0
4,https://openalex.org/W1023696101,This paper presents a model to perform energy ...,"[[https://openalex.org/A5067189491, Arun Verma...",20.0,NaN,"[https://openalex.org/W1965513514, https://ope...","[[process simulation model, 10.1, 0.63], [ener...","[{'id': 'https://openalex.org/C194439259', 'wi...",https://doi.org/10.1016/j.ijhydene.2015.06.149,"[-0.13096152245998383, 0.1287851482629776, 1.0...",...,en,https://openalex.org/W1023696101,2015-09-01,17.861244,Title: Development of a process simulation mod...,Development of a process simulation model for ...,https://doi.org/10.1016/j.ijhydene.2015.06.149,-38.810551,-24.716375,2015.0


In [7]:
df['search_score'] = df['relevance_score'].apply(
    lambda x: [['carbon capture', x]] if pd.notnull(x) else np.nan)

In [9]:
df[df['relevance_score'].isna()].head()

,id,abstract,authors,citationCount,citationCount_normalized,citations,classification_ids,concepts,doi,embedding,...,paperId,publication_date,relevance_score,text,title,url,x,y,year,search_score
9,https://openalex.org/W106711868,"This article outlines the ongoing research, de...",None,1948.0,7.246559,https://api.openalex.org/works?filter=cites:W1...,"[[Microgrids, 6.4, 0.84], [distributed generat...","[{'id': 'https://openalex.org/C2776784348', 'w...",https://doi.org/10.1109/mpae.2007.376583,"[0.014594187028706074, 0.030436156317591667, 0...",...,https://openalex.org/W106711868,None,NaN,Title: Microgrids. Abstract: This article outl...,Microgrids,https://doi.org/10.1109/mpae.2007.376583,-53.935482,-20.382832,2007.0,NaN
18,https://openalex.org/W1120477740,Gelatin methacryloyl (GelMA) hydrogels have be...,None,1570.0,4.870844,https://api.openalex.org/works?filter=cites:W1...,"[[gelatin methacryloyl hydrogels, 3.5, 0.72], ...","[{'id': 'https://openalex.org/C108586683', 'wi...",https://doi.org/10.1016/j.biomaterials.2015.08...,"[-0.7488963007926941, 0.5351231694221497, 0.51...",...,https://openalex.org/W1120477740,None,NaN,"Title: Synthesis, properties, and biomedical a...","Synthesis, properties, and biomedical applicat...",https://doi.org/10.1016/j.biomaterials.2015.08...,-3.964495,7.856454,2015.0,NaN
38,https://openalex.org/W1195955210,The emerging trends in the combinatorial chemi...,None,815.0,0.125699,https://api.openalex.org/works?filter=cites:W1...,"[[insoluble drug delivery, 10.1, 0.6], [combin...","[{'id': 'https://openalex.org/C2780035454', 'w...",https://doi.org/10.1016/j.apsb.2015.07.003,"[-0.21994946897029877, 0.5684394836425781, 0.1...",...,https://openalex.org/W1195955210,None,NaN,Title: Insoluble drug delivery strategies: rev...,Insoluble drug delivery strategies: review of ...,https://doi.org/10.1016/j.apsb.2015.07.003,-20.593531,3.995265,2015.0,NaN
51,https://openalex.org/W136378602,ADVERTISEMENT RETURN TO ISSUEPREVArticleNEXTMe...,None,872.0,0.483942,https://api.openalex.org/works?filter=cites:W1...,"[[methanol electrooxidation, platinum-rutheniu...","[{'id': 'https://openalex.org/C2778805511', 'w...",https://doi.org/10.1021/j100148a030,"[-0.7311024069786072, 0.49849963188171387, 0.3...",...,https://openalex.org/W136378602,None,NaN,Title: Methanol electrooxidation on well-chara...,Methanol electrooxidation on well-characterize...,https://doi.org/10.1021/j100148a030,-59.012787,0.550141,1993.0,NaN
69,https://openalex.org/W1480987708,None,None,836.0,0.257683,https://api.openalex.org/works?filter=cites:W1...,"[[photoactivated γ-secretase inhibitors, activ...","[{'id': 'https://openalex.org/C175344181', 'wi...",https://doi.org/10.1038/35015085,"[0.14470750093460083, 1.0298190116882324, 0.72...",...,https://openalex.org/W1480987708,None,NaN,Title: Photoactivated γ-secretase inhibitors d...,Photoactivated γ-secretase inhibitors directed...,https://doi.org/10.1038/35015085,-12.743648,7.520947,2000.0,NaN


In [10]:
now = datetime.now()
date_str = now.strftime('%y-%m-%d')
time_str = now.strftime('%H-%M-%S')
if not os.path.exists(f'papers/{date_str}'):
    os.makedirs(f'papers/{date_str}')

# save the taxonomy and df to a txt and csv file
note = "added_search_score_column"
df.to_json(f'papers/{date_str}/{time_str}_{df.shape[0]}_added_search_score_column.json', orient='records')
df[['title', 'classification_ids']].to_json(f'papers/{date_str}/{time_str}_{df.shape[0]}_{note}_manual_inspection.json', orient='records', indent=2)

# save to main
df.to_json(f'papers/latest_papers.json', orient='records')